This is a recommender system built using K nearest neighbors

In [14]:
#!/usr/bin/env python
# coding: utf-8

# general purpose libraries
import platform
import os
import dateutil
import itertools

# database connection libraries
import sqlite3
from sqlite3 import Error

# mathematics libraries
import pandas as pd
import numpy as np

# ML libraries
from sklearn.neighbors import KDTree                    # Nearest neighbors
from sklearn.preprocessing import StandardScaler        # scales all values between a range 
from sklearn.model_selection import train_test_split    # cross-validation
from tpot import TPOTRegressor                          # genetic algorithm implementation
from sklearn.externals import joblib                    # model loading
import pickle                                           # model saving

# data visualisation libraries
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

this Function connects with the SQL DB

In [15]:
def create_connection(db_file):
        """ create a database connection to the SQLite database
            specified by the db_file
        :param db_file: database file
        :return: Connection object or None
        """
        try:
            conn = sqlite3.connect(db_file)
            return conn
        except Error as e:
            print(e)
        return None

Connecting all the DBs

In [49]:
Sql_connection1 = create_connection('/opt/apps/scripts/jaarvis_demand_supply/evo.db')
Sql_connection = create_connection('/opt/apps/scripts/jaarvis_demand_supply/zone_travel.db')
Sql_connection2 = create_connection('/opt/apps/scripts/jaarvis_demand_supply/zone_lat_log.db')

Here we are checking what tables are there in our DB

In [17]:
cursor = Sql_connection1.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('vehicles',), ('sqlite_sequence',), ('zones',), ('zone_technician',), ('zone_trip_avg',), ('zone_bound',)]


In [18]:
technician_db = pd.read_sql_query("select * from zone_technician;", Sql_connection1)
technician_db = technician_db.fillna(-9999)

In [19]:
avg_trip_cost = pd.read_sql_query("select * from zone_trip_avg;", Sql_connection1)


In [20]:
zone_detail_df = pd.read_sql_query("select * from zone_travel_detail;", Sql_connection)
# zone_detail_df.head()

Data Preprocessing

In [21]:
temp = zone_detail_df.loc[zone_detail_df['to_zone'].isin(list(avg_trip_cost.zone.unique()))]
temp = temp.loc[temp['from_zone'].isin(list(avg_trip_cost.zone.unique()))]

zone_detail_df = temp

In [22]:
avg_trip_cost.total_trips = avg_trip_cost.total_trips.astype(int)

In [23]:
avg_trip_cost = avg_trip_cost.sort_values(by = 'zone')

In [24]:
# bringing all DFs in same format so that they can be joined to pass to the ML model
cost_diff=pd.DataFrame(list(itertools.product(avg_trip_cost.zone,avg_trip_cost.zone)),columns=['from_zone','to_zone'])
dic = dict(zip(avg_trip_cost.zone,avg_trip_cost.avg_cost))
cost_diff['cost_diff']=cost_diff.from_zone.map(dic)-cost_diff.to_zone.map(dic)
cost_diff.head()

cost_diff = cost_diff.fillna(-9999)

In [25]:
# Missing values handling
technician_db = technician_db.loc[technician_db['zone'].isin(list(avg_trip_cost.zone.unique()))]

In [26]:
technician_db.avg.loc[technician_db['avg'] < 3 ] = -15
technician_db.avg.loc[technician_db['avg'] > 10] = -1
technician_db.avg.loc[technician_db['avg'] > 3] = 10
technician_db.avg.loc[technician_db['avg'] == -1] = 0

technician_db = technician_db.fillna(-9999)

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [27]:
# missing_zones = pd.DataFrame({'id':[],'avg':[],'zone':[],'from_time':[],'to_time':[]})
# missing_zones.zone= avg_trip_cost.zone[~avg_trip_cost.zone.isin(technician_db.zone)]

In [28]:
# temp_df = pd.concat([technician_db,missing_zones],ignore_index=True)

In [29]:
technician_db = technician_db.drop_duplicates('zone', keep='last')

In [30]:
import itertools
tech_diff=pd.DataFrame(list(itertools.product(technician_db.zone,technician_db.zone)),columns=['from_zone','to_zone'])
dic = dict(zip(technician_db.zone,technician_db.avg))
tech_diff['tech_diff']=tech_diff.from_zone.map(dic)

tech_diff = tech_diff.fillna(-9999)



In [31]:
# final DF after data preprocessing & cleaning & joining
supply_df = pd.merge(zone_detail_df, tech_diff,  how='left', left_on=['from_zone','to_zone'], right_on = ['from_zone','to_zone'])
supply_df = pd.merge(supply_df,cost_diff ,  how='left', left_on=['from_zone','to_zone'], right_on = ['from_zone','to_zone'])
supply_df = supply_df.fillna(-9999)

In [32]:
supply_df['vector'] = list(map(list,(zip(supply_df.kilometers,supply_df.minutes,supply_df.cost_diff,supply_df.tech_diff))))

In [33]:
supply_pivot_df = pd.pivot_table(supply_df,values=['vector'],index=['from_zone'],columns=['to_zone'],aggfunc='first')

In [34]:
supply_pivot_df = supply_pivot_df.vector

In [35]:
supply_pivot_df = supply_pivot_df.sort_index()
supply_pivot_df = supply_pivot_df[supply_pivot_df.columns.sort_values()]


In [36]:
supply_pivot_df=supply_pivot_df.applymap(lambda x: [-9999,-9999,-9999,-9999] if x is np.nan else x)
supply_pivot_df = supply_pivot_df[supply_pivot_df.index.isin(supply_pivot_df.columns)]
supply_pivot_df = supply_pivot_df.loc[:, supply_pivot_df.columns.isin(supply_pivot_df.index)]

In [37]:
for i in range(supply_pivot_df.shape[0]):
    supply_pivot_df.iloc[i,i]=[0,0,0,0]

weighted_score = time*x1+distance*x2+average_trip_cost*x3+no_of_tech*x4+cost_of_relocating*x5+hot_zone*x6+supply gap*x7

Garstin_Bastion = 

In [59]:
import json

# def nearest_neighbors():
from sklearn.neighbors import KDTree
from collections import defaultdict
neighbor_dict=defaultdict(dict)
for x in range(0,supply_pivot_df.shape[0]):
    df  = [np.array(j) for i,j in supply_pivot_df.iloc[x,:].iteritems()]
    kdt = KDTree(df, leaf_size=30, metric='euclidean')
    zone_neighbors = kdt.query(df, k=10, return_distance=False)[x]
    neighbor_dict[supply_pivot_df.columns[x]]['supply'] = list([supply_pivot_df.columns[i] for i in zone_neighbors])
    neighbor_dict[supply_pivot_df.columns[x]]['lat_long'] = lat_long_df[lat_long_df['zone'].str.contains(supply_pivot_df.columns[x])]['lat'].values[0], \
                                           lat_long_df[lat_long_df['zone'].str.contains(supply_pivot_df.columns[x])]['log'].values[0]

data = json.dumps(neighbor_dict)
with open('supply.json', 'w') as f:
            f.write(data)

In [341]:
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
# import numpy as np
# # X, Y, Z, U, V, W = zip(*df)
# # fig = plt.figure()
# # ax = fig.add_subplot(111, projection='3d')
# # ax.quiver(X, Y, Z, U, V, W)

# # ax.set_xlim([-2, 2])
# # ax.set_ylim([-2, 2])
# # ax.set_zlim([-2, 2])
# # ax.legend()
# # plt.show()

# avg_trip_cost = avg_trip_cost[2:]
# # np.where(avg_trip_cost.zone =='V5Z 2L5')
# # df = avg_trip_cost.set_index('zone')
# # mux = pd.MultiIndex.from_product([avg_trip_cost.zone, avg_trip_cost.zone], names=['name1','name2'])

# # df = (avg_trip_cost['avg_cost'].reindex(mux, level=0)
# #         .sub(df.reindex(mux, level=1)['avg_cost'])
# #         .rename('time_diff')
# #         .reset_index())
# supply_df = supply_df[numpy.isfinite(supply_df['cost_diff'])]
# supply_df = supply_df[numpy.isfinite(supply_df['kilometers'])]
# list(set(supply_df.to_zone) - set(supply_df.from_zone))
# # zone_detail_df.to_zone.isin()
len(set(avg_trip_cost.zone.unique()).difference((set(zone_detail_df.to_zone.unique()))))


30

In [286]:
# demand = pd.read_json('http://139.59.93.34:5000/demand')
# live_cars = pd.read_json('live_cars.json')
# demand = demand.T
# live_cars = live_cars.T
# # demand = demand.fillna(0)
# # live_cars = live_cars.fillna(0)

# def type_conversion(x):
#     if type(x)== list:
#         return tuple(x)
#     else:
#         print(x)
    
# live_cars.lat_long = live_cars.lat_long.apply(lambda x: type_conversion(x))
# demand.lat_long = demand.lat_long.apply(lambda x: type_conversion(x))

# result= pd.merge(live_cars,demand, how='outer',on='lat_long')



nan


In [ ]:
#!/usr/bin/env python
# coding: utf-8

"""recommender system built on euclidean distance and K-nearest neighbors"""

# general purpose libraries
import platform
import os
import dateutil
import itertools

# database connection libraries
import sqlite3
from sqlite3 import Error

# mathematics libraries
import pandas as pd
import numpy as np

# ML libraries
from sklearn.neighbors import KDTree                    # Nearest neighbors
from sklearn.preprocessing import StandardScaler        # scales all values between a range 
from sklearn.model_selection import train_test_split    # cross-validation
from tpot import TPOTRegressor                          # genetic algorithm implementation
from sklearn.externals import joblib                    # model loading
import pickle                                           # model saving

# data visualisation libraries
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# serializing libraries
import json

class SupplyRecommender():
    
    def __init__(self):
        self.Sql_connection1 = self.create_connection('/opt/apps/scripts/jaarvis_demand_supply/evo.db')
        self.Sql_connection = self.create_connection('/opt/apps/scripts/jaarvis_demand_supply/zone_travel.db')
        self.Sql_connection2 = self.create_connection('/opt/apps/scripts/jaarvis_demand_supply/zone_lat_log.db')
        

    def create_connection(self,db_file):
        """ create a database connection to the SQLite database
            specified by the db_file
        :param db_file: database file
        :return: Connection object or None
        """
        try:
            conn = sqlite3.connect(db_file)
            return conn
        except Error as e:
            print(e)
        return None
    
    def getTechnicians(self):

        technician_db = pd.read_sql_query("select * from zone_technician;", self.Sql_connection1)
        technician_db = technician_db.fillna(-9999)
        
        # Missing values handling
        technician_db = technician_db.loc[technician_db['zone'].isin(list(avg_trip_cost.zone.unique()))]
        
        # Scaling values in range (-9999,10)
        technician_db.avg.loc[technician_db['avg'] < 3 ] = -15
        technician_db.avg.loc[technician_db['avg'] > 10] = -1
        technician_db.avg.loc[technician_db['avg'] > 3] = 10
        technician_db.avg.loc[technician_db['avg'] == -1] = 0
        technician_db = technician_db.fillna(-9999)

        
        technician_db = technician_db.drop_duplicates('zone', keep='last')
        
        # calculating technicians in zones
        tech_diff=pd.DataFrame(list(itertools.product(technician_db.zone,technician_db.zone)),columns=['from_zone','to_zone'])
        dic = dict(zip(technician_db.zone,technician_db.avg))
        tech_diff['tech_diff']=tech_diff.from_zone.map(dic)

        tech_diff = tech_diff.fillna(-9999)
        
        return tech_diff

    # cursor = Sql_connection1.cursor()
    # cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    # print(cursor.fetchall())

    def getAvgTripCost(self):
        
        avg_trip_cost = pd.read_sql_query("select * from zone_trip_avg;", self.Sql_connection1)
        avg_trip_cost.total_trips = avg_trip_cost.total_trips.astype(int)
        avg_trip_cost = avg_trip_cost.sort_values(by = 'zone')

        cost_diff=pd.DataFrame(list(itertools.product(avg_trip_cost.zone,avg_trip_cost.zone)),columns=['from_zone','to_zone'])
        dic = dict(zip(avg_trip_cost.zone,avg_trip_cost.avg_cost))
        cost_diff['cost_diff']=cost_diff.from_zone.map(dic)-cost_diff.to_zone.map(dic)
        cost_diff.head()

        cost_diff = cost_diff.fillna(-9999)

        return cost_diff

    def getDistanceTime(self):
        
        zone_detail_df = pd.read_sql_query("select * from zone_travel_detail;", self.Sql_connection)
        # zone_detail_df.head()

        temp = zone_detail_df.loc[zone_detail_df['to_zone'].isin(list(avg_trip_cost.zone.unique()))]
        temp = temp.loc[temp['from_zone'].isin(list(avg_trip_cost.zone.unique()))]

        zone_detail_df = temp
        
        return zone_detail_df

# bringing all DFs in same format so that they can be joined to pass to the ML model

    def supplyData(self):
        
        zone_detail_df = self.getDistanceTime()
        tech_diff = self.getTechnicians()
        cost_diff = self.getAvgTripCost
        
        # final DF after data preprocessing & cleaning & joining
        supply_df = pd.merge(zone_detail_df, tech_diff,  how='left', left_on=['from_zone','to_zone'], right_on = ['from_zone','to_zone'])
        supply_df = pd.merge(supply_df,cost_diff ,  how='left', left_on=['from_zone','to_zone'], right_on = ['from_zone','to_zone'])
        supply_df = supply_df.fillna(-9999)

        supply_df['vector'] = list(map(list,(zip(supply_df.kilometers,supply_df.minutes,supply_df.cost_diff,supply_df.tech_diff))))

        supply_pivot_df = pd.pivot_table(supply_df,values=['vector'],index=['from_zone'],columns=['to_zone'],aggfunc='first')

        supply_pivot_df = supply_pivot_df.vector

        supply_pivot_df = supply_pivot_df.sort_index()
        supply_pivot_df = supply_pivot_df[supply_pivot_df.columns.sort_values()]

        supply_pivot_df=supply_pivot_df.applymap(lambda x: [-9999,-9999,-9999,-9999] if x is np.nan else x)
        supply_pivot_df = supply_pivot_df[supply_pivot_df.index.isin(supply_pivot_df.columns)]
        supply_pivot_df = supply_pivot_df.loc[:, supply_pivot_df.columns.isin(supply_pivot_df.index)]

        for i in range(supply_pivot_df.shape[0]):
            supply_pivot_df.iloc[i,i]=[0,0,0,0]

        return supply_pivot_df
    
    def nearest_neighbors():
        supply_pivot_df = self.supplyData()
        
        from sklearn.neighbors import KDTree
        from collections import defaultdict
        neighbor_dict=defaultdict(dict)
        for x in range(0,supply_pivot_df.shape[0]):
            df  = [np.array(j) for i,j in supply_pivot_df.iloc[x,:].iteritems()]
            kdt = KDTree(df, leaf_size=30, metric='euclidean')
            zone_neighbors = kdt.query(df, k=10, return_distance=False)[x]
            neighbor_dict[supply_pivot_df.columns[x]]['supply'] = list([supply_pivot_df.columns[i] for i in zone_neighbors])
            neighbor_dict[supply_pivot_df.columns[x]]['lat_long'] = lat_long_df[lat_long_df['zone'].str.contains(supply_pivot_df.columns[x])]['lat'].values[0], \
                                                   lat_long_df[lat_long_df['zone'].str.contains(supply_pivot_df.columns[x])]['log'].values[0]
    
        data = json.dumps(neighbor_dict)
        with open('supply.json', 'w') as f:
                    f.write(data)

In [62]:
import os
os.getcwd()

'/opt/apps/Jaarvis_Demand_Supply/available_models'